### Handling PDF  Challenges 
PDFs are hard to parse because they:

-> store text in complex ways (not just simple text)

-> can have formatting issues

-> may contain scanned images(OCR)

-> Often have extraction artifacts

In [ ]:
# Example of raw PDF extraction

raw_pdf_text = """ Company financial report

        The financial report for the year 2023 shows a significant increase in
        revenue compared to the previous year. The company has implemented several new
        strategies that have contributed to this growth. The net profit margin has also improved,
        
        
        indicating better cost management and operational efficiency. Overall, the financial performance
        
        
of the company in 2023 has been very positive,
and it is expected to continue this trend in the coming years.

Page 1 of 10
"""
# Apply the cleaning function 
def clean_text(text):
    